# **RAG evaluation**
> - RAG 시스템의 정보 검색 능력과 답변 생성 능력을 측정

> - **평가 지표 (값이 1에 가까울 수록 성능이 좋음)**
> - **LLMContextRecall**: 컨텍스트를 기반으로 정보를 재현한 능력을 측정
> - **LLMContextPrecisionWithReference**: 문서의 정확하고 관련된 정보를 기반으로 답변을 생성했는지 평가
> - **Faithfulness**: 문서의 내용에 얼마나 충실하게 답변했는지 평가
> - **AnswerRelevancy**: 질문과 답변 간의 의미적 관련성을 측정

In [1]:
from langchain_chroma import Chroma

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate, MessagesPlaceholder, ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser

from langchain_core.runnables import RunnableLambda
from langchain_core.documents import Document
from langchain_community.tools import TavilySearchResults
from langchain_core.runnables.history import RunnableWithMessageHistory, RunnableLambda

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from pydantic import BaseModel, Field

from textwrap import dedent
from operator import itemgetter
from pprint import pprint
import json
import random
import re

from langchain_core.tools import tool
from itertools import chain

from dotenv import load_dotenv
load_dotenv()


True

# **질문-답변 생성**

- 평가 데이터로 사용할 context 100개를 랜덤으로 추출한 뒤, 각 3개씩의 질문-답변 쌍을 생성하여 **총 300개의 질문-답변 쌍**을 생성함
- 생성된 데이터는 csv 파일 형태로 저장

In [ ]:
# 데이터 파일 경로
DOC_PATH = 'policy_result.json'

# 파일 로딩
with open(DOC_PATH, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# 문서 리스트 생성 (각 정책 항목을 문자열로 변환)
docs = []
for policy_name, details in json_data.items():
    # 정책 이름과 상세 내용을 문자열로 합침
    policy_content = f"정책 이름: {policy_name}\n"
    for detail in details:
        policy_content += f"{detail}\n"
    docs.append(policy_content.strip())

# docs 내용 확인
len(docs), docs[:1]  # 문서 개수와 첫 번째 문서 내용 출력

(1268,
 ['정책 이름: 청년 주택드림 청약통장\n기관:국토교통부\n정책 분야:주거분야\n지원 내용:  이율  최대 4.5    소득   5 000만원 직전년도 신고소득이      무주택   납입한도  월 100만원   납입금액의 40 까지 소득공제   주택청약에 당첨된  청년주택드림 대출 계   청년주택드림 청약통장 1년 이상 가입   1 000만원 이상 납입   당첨  분양가 80 까지 대출   금리 최저  2.2  만기  소득별 차  최장 40년까지 지원 고정금리    6억 이하  전용면적 85제곱미터 이하 주택   생애주기별 금리 인하  결혼 0.1 p 인하  출산  0.5 p 인하  다녀  0.2 p 인하\n사업 신청 기간:2024년 02월21일   2024년 12월31일\n비고: 기존 청년우대형통장 가입는 출일에 맞춰서 동 전환 일반청약 가입는 요건에만 맞으면 전환 가입 \n연령:만 19세   34세\n거주지 및 소득:  청년 주택드림 청약통장 소득    5 000만원 이하\n기타 유익 정보:  업무 취급 은행   우리은행   1599 0800   KB 국민은행   1599 1771   IBK 기업은행   1566 2566   NHBank   1588 2100   신한은행   1599 8000   하나은행   1599 1111   대구은행   1566 5050   부산은행   1800 1333   경남은행   1600 8585\n주관 기관:국토교통부\n운영 기관:국토교통부'])

In [ ]:
### 평가 데이터로 사용할 context 추출
total_samples = 100

# index shuffle 후 total_samples만큼 context 추출
idx_list = list(range(len(docs)))
random.shuffle(idx_list)

eval_context_list = []
while len(eval_context_list) < total_samples:
    idx = idx_list.pop()
    context = docs[idx]
    if len(context) > 50: # 50글자 이상인 text만 사용
        eval_context_list.append(context)

len(eval_context_list)

In [24]:
# user_input: 질문
# ####### retrieved_contexts: 검색된 문서의 내용(page_content)들
# qa_context: 질문 답변 쌍을 만들 때 참고한 context
        # retrieved_contexts: 검색된 문서의 내용은 실제 RAG 실행시 넣는다.
        # response: 모델의 답변 - 실제 RAG 실행시 넣는다.
# reference: 정답

#######################################################################################################################################

class EvalDatasetSchema(BaseModel):
    user_input: str = Field(..., title="질문(question)")
    qa_context: list[str] = Field(..., title="질문-답변 쌍을 만들 때 참조한 context.")
    reference: str = Field(..., title="질문의 정답(ground truth)")

parser = JsonOutputParser(pydantic_object=EvalDatasetSchema)

eval_model = ChatOpenAI(model="gpt-4o")

prompt_template = PromptTemplate.from_template(
    template=dedent("""
        당신은 RAG 평가를 위해 질문과 정답 쌍을 생성하는 인공지능 비서입니다.
        해당 RAG는 사용자에게 각 정책에 대한 질의응답을 위해 만들어졌으니 이를 참고하여 평가 질문과 정답 쌍을 생성하세요.
        주어진 [Context]를 활용해서 최대한 다양한 질문-정답 쌍을 만들어 주세요.
        다음 [Context] 에 문서가 주어지면 해당 문서를 기반으로 {num_questions}개 질문-정답 쌍을 생성하세요.

        질문과 정답을 생성한 후 아래의 출력 형식 GUIDE 에 맞게 생성합니다.
        질문은 반드시 [Context] 문서에 있는 정보를 바탕으로 생성해야 합니다. [Context]에 없는 내용을 가지고 질문-정답을 절대 만들면 안됩니다.
        질문은 간결하게 작성합니다.

        질문을 만들 때 반드시 각 [Context] 문서의 "정책 이름"을 질문에 포함해야합니다.
        예시는 "경남 저소득 증장애인 집정리 범사업을 주관하는 기관은 어디인가요?" 라는 형식의 질문 생성입니다.

        하나의 질문에는 한 가지의 내용만 작성합니다.
        질문을 만들 때 "제공된 문맥에서", "문서에 설명된 대로", "주어진 문서에 따라" 또는 이와 유사한 말을 하지 마세요.
        정답은 반드시 [Context]에 있는 정보를 바탕으로 작성합니다. 없는 내용을 추가하지 않습니다.
        질문과 정답을 만들고 그 내용이 [Context] 에 있는 항목인지 다시 한번 확인합니다.

        어떤 형식으로 만들었든 생성된 질문-답변 쌍은 반드시 dictionary 형태로 정의하고 list로 묶어서 반환해야 합니다.
        어떤 형식으로 만들었든 질문-답변 쌍은 반드시 {num_questions}개를 만들어야 합니다.

        출력 형식: {format_instructions}

        [Context]
        {context}
        """
    ),
    partial_variables={"format_instructions":parser.get_format_instructions()}
)
# print(prompt_template.template)

eval_dataset_generator = prompt_template | eval_model | parser

In [25]:
############################################################
# eval_context_list 로 만들기
############################################################
eval_data_list = []
num_questions = 3
for context in eval_context_list:
    _eval_data_list = eval_dataset_generator.invoke({"context":context, "num_questions":num_questions})
    eval_data_list.extend(_eval_data_list)

In [ ]:
import pandas as pd
eval_df = pd.DataFrame(eval_data_list)
eval_df.shape

In [27]:
## 생성 된 질문/답 쌍 확인
eval_df

,user_input,qa_context,reference
0,청년 커리어 고민 솔루션 ALL Question 안양의 지원 내용을 알려주세요.,"[정책 이름: 청년 커리어 고민 솔루션 ALL Question 안양, 지원 내...","청년의 고민을 현직가가 답해주는 온라인 멘토링을 제공합니다. 진로, 대학, 취업, ..."
1,청년 커리어 고민 솔루션 ALL Question 안양의 신청 절차는 어떻게 되나요?,"[정책 이름: 청년 커리어 고민 솔루션 ALL Question 안양, 신청 절...","16세부터 18세는 안양청년광장 회원가입 후 ALL Q를 이용하고, 19세부터 39..."
2,청년 커리어 고민 솔루션 ALL Question 안양의 운영 기간은 언제부터인가요?,"[정책 이름: 청년 커리어 고민 솔루션 ALL Question 안양, 비고: ...",2024년 3월 4일부터 계속 운영됩니다.
3,경북 영덕군 AIDS 환 의료비 지원 정책의 주관 기관은 어디인가요?,"[정책 이름: 경북 영덕군 AIDS 환 의료비 지원, 기관:경북 영덕군, 정책 분야...",영덕군청 감염병관리팀
4,경북 영덕군 AIDS 환 의료비 지원 정책의 신청 절차는 어떻게 되나요?,"[정책 이름: 경북 영덕군 AIDS 환 의료비 지원, 기관:경북 영덕군, 정책 분야...",본인 서식지 작성
5,경북 영덕군 AIDS 환 의료비 지원 정책의 운영 기관은 어디인가요?,"[정책 이름: 경북 영덕군 AIDS 환 의료비 지원, 기관:경북 영덕군, 정책 분야...",영덕군청 감염병관리팀
6,의정부 청년 정신건강상담의 지원 내용은 무엇인가요?,"[정책 이름: 의정부 청년 정신건강상담, 지원 내용: 내용 청년센터에서 진행되는 청...","청년센터에서 진행되는 청년정신건강상담이며, 청년들의 정신건강문제를 조기 발견하고 상..."
7,의정부 청년 정신건강상담의 주관 기관은 어디인가요?,"[정책 이름: 의정부 청년 정신건강상담, 주관 기관: 의정부 청년센터 031 828...",의정부 청년센터
8,의정부 청년 정신건강상담에 신청할 수 있는 연령대는 어떻게 되나요?,"[정책 이름: 의정부 청년 정신건강상담, 연령: 만 19세 34세]",만 19세에서 34세


In [28]:
# csv 파일로 저장
output_path = 'eval_question_sample.csv'    # 저장 경로
eval_df.to_csv(output_path, index=False, encoding='utf-8')

# 저장된 파일 확인
print(f"CSV 파일이 {output_path}에 저장되었습니다.")

CSV 파일이 eval_question_sample.csv에 저장되었습니다.


# **Chain 구성**
- Vector Store와 web에서 검색한 context들과 RAG 시스템의 응답이 출력되도록 chain을 구성

In [ ]:
# 저장된 파일을 불러오기
## 앞선 코드에서 이어서 진행 중일 경우 실행하지 않아도 됨.

import pandas as pd

# CSV 파일 경로
csv_file_path = "eval_question.csv"

# CSV 파일 읽기
eval_df = pd.read_csv(csv_file_path)

# 데이터 확인
print(eval_df.head())  # 상위 5개의 데이터 출력
print(eval_df.info())  # 데이터프레임 정보 확인

                                 user_input  \
0         울산 동구 지역산업 청년일리 사업의 주관 기관은 어디인가요?   
1         울산 동구 지역산업 청년일리 사업의 지원 내용은 무엇인가요?   
2  울산 동구 지역산업 청년일리 사업의 신청 가능한 연령대는 어떻게 되나요?   
3       청년지원센터 더누림 플랫폼 운영 광주의 주관 기관은 어디인가요?   
4     청년지원센터 더누림 플랫폼 운영 광주의 신청 절차는 어떻게 되나요?   

                                          qa_context  \
0  ['정책 이름: 울산 동구 지역산업 청년일리 사업', '주관 기관:울산광역 동구 일...   
1  ['정책 이름: 울산 동구 지역산업 청년일리 사업', '지원 내용: 추진방법   울...   
2    ['정책 이름: 울산 동구 지역산업 청년일리 사업', '연령:만 18세   39세']   
3   ['정책 이름: 청년지원센터 더누림 플랫폼 운영 광주', '주관 기관: 경기도 광주']   
4  ['정책 이름: 청년지원센터 더누림 플랫폼 운영 광주', '신청 절차: 광주 청년지...   

                                           reference  
0                                울산광역 동구 일리정책과 일리정책팀  
1  지역의 산업에 맞는 인력양성과 취업역량 강화 교육, 원활한 구인 구직 매칭을 지원합니다.  
2                                  만 18세부터 39세까지입니다.  
3                                             경기도 광주  
4                                   광주 청년지원센터 홈페이지 접  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 

In [30]:
    # Vector Store
    embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")  # Vector DB와 동일한 1536 차원 모델 사용
    COLLECTION_NAME = "policy"
    PERSIST_DIRECTORY = "data/vector_store/policy"

    # vector store 연결
    vector_store = Chroma(
        embedding_function=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )

In [31]:
# 평가하려는 RAG 시스템

from operator import itemgetter

# Retriever 설정 - 검색 설정
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,
        "fetch_k": 10,
        "lambda_mult": 0.2
    }
)

# LLM 구성
messages = [
            ("ai", """
            당신은 유능한 청년지원정책 추천 전문 AI 챗봇입니다.
            주요 목표는 사용자의 요청에 따라 알맞는 청년지원정책을 추천하는 것입니다.
             
            다음은 답변을 작성하기 위한 지침(guidelines)입니다:
            1. 주어진 context(데이터 및 검색 결과)를 바탕으로만 대답해주세요.
            2. 모든 답변은 학습된 정책 데이터를 바탕으로 반드시 사용자가 물어본 질문에 대한 정확한 정보만 작성하세요.
            3. 예를 들어, 사용자가 "서울시 주거 정책을 알려줘" 라고 질문했다면 "서울 시민"이 지원대상이 되는 답변만을 생성해야하며 "고양시"나 "경남" 지역에서 시행 중인 정책을 답변으로 가져와서는 안됩니다.
            4. 답변에 불필요한 정보는 제공하지 마세요.
            5. 해당 데이터에 없는 내용은 검색해서 대답세요. 다만, 검색 도구(TavilySearch 등)에서도 찾을 수 없는 경우, 답변을 추측하거나 임의로 생성하지 말고 "잘 모르겠습니다."라고 답변하세요.검색으로도 정보를 찾을 수 없을 경우 답변을 추측하거나 임의로 생성하지말고, "잘 모르겠습니다."라고 답변하세요.
            6. 답변은 체계적이고, 비전문가 사용자도 이해하기 쉽게 답변을 작성하세요.
            7. 항상 최신의 정확한 정보를 제공하기 위해 노력하세요.
            8. 질문을 완전히 이해하지 못할 경우, 구체적인 질문을 다시 받을 수 있도록 사용자에게 유도 질문을 하세요.     
            9. 답변 스타일은 간결하고 논리적으로 작성하세요. 필요시, 리스트 형식으로 정리하세요.
            10. 답변이 사용자가 질문한 내용에 부합하는지 다시 한번 확인하세요.
            11. 질문을 만들 때 "제공된 문맥에서", "문서에 설명된 대로", "주어진 문서에 따라" 또는 이와 유사한 말을 하지 마세요.
            
            위 지침을 따라 사용자의 요청에 맞는 적절한 청년지원정책 정보를 제공합니다.

            {context}")"""),
            MessagesPlaceholder("history"), 
            ("human", "{question}")
]

prompt_template = ChatPromptTemplate(messages)
model = ChatOpenAI(model='gpt-4o', temperature=0)

query = "user_input"

# search_policy tool 정의
@tool
def search_policy(query:str) -> list[Document]:
    """
    Vector Store에 저장된 청년 지원 정책과 해당 정책의 정보를 검색한다.
    이 도구는 청년 지원 정책 관련 질문에 대해 실행한다.
    """
    result = retriever.invoke(query) 
    if len(result): 
        return result
    else:
        return [Document(page_content = "검색 결과가 없습니다.")]
    
    # TavilySearch web 검색
tavily_search = TavilySearchResults(max_results=2)

# 최종 응답 생성
def generate_final_input(query, result_from_db, result_from_web):
    combined_context = [
        f"저장된 데이터에서 찾은 정보:\n{result_from_db}\n",
        "실시간 웹 검색에서 확인된 정보:\n"
    ]

    if isinstance(result_from_web, list):
        combined_context.extend(
            [f"[{idx}] {result.get('title', '제목 없음')}: {result.get('content', '내용 없음')}"
             for idx, result in enumerate(result_from_web, start=1)]
        )
    else:
        combined_context.append("웹 검색 결과를 처리할 수 없습니다.")
    
    combined_context = "\n".join(combined_context)
    return f"{combined_context}를 참고해서 {query}에 대한 답변 생성"


# 실행에 필요한 인풋 설정
runnable = {
    'context': RunnableLambda(lambda x: retriever.invoke(x['question'])),
    'question': itemgetter("question"),
    'history': itemgetter('history')
} | prompt_template | model | StrOutputParser()

# chain 설정
chain = RunnableWithMessageHistory(
    runnable=runnable,
    get_session_history=lambda session_id: memory.chat_memory,
    input_messages_key="question",
    history_messages_key="history"
)

# DB 검색
result_from_db = search_policy(query)

# Tavily 검색(web 검색)
try:
    result_from_web = tavily_search.invoke(query)
    if not result_from_web or not isinstance(result_from_web, list):
        raise ValueError("유효하지 않은 Tavily 검색 결과")
except Exception as e:
    result_from_web = [{"title": "오류 발생", "content": str(e)}]

# 최종 응답 생성
final_model = ChatOpenAI(model="gpt-4o-mini")
final_input = generate_final_input(query, result_from_db, result_from_web)
final_response = final_model.invoke(final_input)

In [ ]:
# context들과 모델답변을 응답 받아 eval_dataset(eval_df)에 추가.

context_list = []
response_list = []

for user_input in eval_df['user_input']:
    # 정책 검색
    result_from_db = search_policy.invoke(user_input)
    result_from_web = tavily_search.invoke(user_input)
    
    final_input = generate_final_input(user_input, result_from_db, result_from_web)
    final_response = final_model.invoke(final_input)
    
    context = [f"{result_from_db}, {result_from_web}"]
    context_list.append(context)
    response_list.append(final_response.content)    # content만 response_list에 저장

In [33]:
# 확인
response_list
# context_list

[AIMessage(content='울산 동구 지역산업맞춤형 청년일자리 사업의 주관 기관은 울산광역시 동구 일자리정책과입니다. 이 사업의 주요 내용은 지역 산업 수요에 맞는 인력 양성과 취업 역량 강화 교육을 통해 지역 고용 문제를 해결하는 것입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 3085, 'total_tokens': 3154, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-078e4b0b-2461-436c-a82e-7062bb65d04d-0', usage_metadata={'input_tokens': 3085, 'output_tokens': 69, 'total_tokens': 3154, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='울산 동구 지역산업맞춤형 청년일자리 사업의 지원 내용은 다음과 같습니다:\n\n1. **사업 목표**: 양질의 일자리 창출을 통해 지역 경제를 활성화하고, 청년 인구 유입을 통해 도시 경쟁력을 강화하는 것을 목표로 합니다.\n

In [ ]:
# context 추가
eval_df["retrieved_contexts"] = context_list

# 정답 추가
eval_df["response"] = response_list

# csv 파일로 저장
output_path = 'finaaaaal_question.csv'
eval_df.to_csv(output_path, index=False, encoding='utf-8')

# 저장된 파일 확인
print(f"CSV 파일이 {output_path}에 저장되었습니다.")

CSV 파일이 finaaaaal_question.csv에 저장되었습니다.


## 평가

In [6]:
# 위의 코드를 연속적으로 실행 중인 경우엔 해당 코드를 실행하지 않아도 됨.
## csv로 저장된 파일을 불러와서 평가를 진행할 경우에만 실행.

import pandas as pd
import ast

# CSV 파일 경로
csv_file_path = "finaaaaal_question.csv"

# CSV 파일 읽기
eval_df = pd.read_csv(csv_file_path)

### CSV 파일을 불러올 경우, 'retrieved_contexts' 항목이 문자열이기 때문에 올바른 데이터 유형인 LIST로 변경해주어야 함.
eval_df['retrieved_contexts'] = eval_df['retrieved_contexts'].apply(ast.literal_eval)

# Dataframe으로 부터 EvalDataset 생성
eval_dataset = EvaluationDataset.from_pandas(eval_df)
eval_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=300)

In [7]:
model_name = "gpt-4o"
model = ChatOpenAI(model=model_name)
eval_llm = LangchainLLMWrapper(model)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)

In [8]:
# 분당 토큰 리미트으로 평가별 나누어 실행
run_config = RunConfig(
    timeout=360,
    max_retries=20,
    max_wait=360,
    max_workers=1
)

metrics1 = [
    LLMContextRecall(llm=eval_llm),
]
metrics2 = [
    LLMContextPrecisionWithReference(llm=eval_llm),
]
metrics3 = [
    Faithfulness(llm=eval_llm),
]
metrics4 = [
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)
]

In [ ]:
result1 = evaluate(dataset=eval_dataset, metrics=metrics1, run_config=run_config, batch_size=5)
result1

In [ ]:
result2 = evaluate(dataset=eval_dataset, metrics=metrics2, run_config=run_config, batch_size=5)
result2

In [ ]:
result3 = evaluate(dataset=eval_dataset, metrics=metrics3, run_config=run_config, batch_size=5)
result3

In [ ]:
result4 = evaluate(dataset=eval_dataset, metrics=metrics4, run_config=run_config, batch_size=5)
result4